In [1]:
#imports block

import pandas as pd
import numpy as np
import string, re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import TfidfVectorizer
#Models
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
#from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, log_loss, accuracy_score

import nltk
import spacy
#nltk.download()
from nltk import word_tokenize, pos_tag
#importing the Stemming function from nltk library
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tqdm.notebook import tqdm
tqdm.pandas()

stop_words = stopwords.words('english')
nlp = spacy.load('en_core_web_sm')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [20]:
#Reading Input
fake_df = pd.read_csv('Fake.csv')
true_df = pd.read_csv('True.csv')

In [21]:
true_df.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [22]:
fake_df.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,31-Dec-17
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,31-Dec-17
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,30-Dec-17
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,29-Dec-17
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,25-Dec-17


In [23]:
#Exploratory Data Analysis

In [24]:
fake_df['class'] = 0
true_df['class'] = 1
#Concat Full Data
df = pd.concat([fake_df, true_df])
#Shuffle the Data
df = df.sample(frac = 1)
df.reset_index(inplace = True)
#Filter Required columns
df= df[['text','class']]
df

,text,class
0,WASHINGTON (Reuters) - The U.S. Senate will vo...,1
1,"Trump s transition is humming right along now,...",0
2,Spewing falsehoods has become the favorite pas...,0
3,I feel like I ve been drugged by Obama s rhet...,0
4,Ryan has never made any secret about his desir...,0
...,...,...
44935,STRASBOURG (Reuters) - European Commission chi...,1
44936,Just as they did during and in the aftermath o...,0
44937,An Asian student reporter on assignment for ES...,0
44938,Why should armed civilians have to protect and...,0


In [25]:
#Train Test Split
#Text Processing pipeline: Lower case + remove punctuation + Tokenization + Lemmatization + Stop words Removal
#Convert these to Vectors using BOW or TF-IDF or Embeddings(Glove, fasttext, Word2Vec)
#Make Baseline ML models(Log Reg, SVM, Naive Bayes, Decision Trees, RF, XGBoost Classifier)
#Hyperparameter Tuning of above models
#Make DL models using CNN, LSTM's, BERT/RoBERTa

Normalisation Required??? 

Normalisation performs:

    Converting dates to text
    Numbers to text
    Currency/Percent signs to text
    Expanding of abbreviations (content dependent) NLP - Natural Language Processing, Neuro-linguistic programming, Non-Linear programming
    Spelling mistakes correction

In [26]:
#from normalise.normalisation import normalise

#def normalization(tokens):
#    user_abbr = { 'NLP' : 'Natural Language Processing'}
#    normalised_tokens = normalise(tokens, user_abbrevs = user_abbr, verbose = False)
#    return normalised_tokens

def remove_stop_words(words):
    filtered_words = [word for word in words if word not in stop_words]
    return filtered_words

def stemming(words):
    porter = PorterStemmer()
    stemmed_words = [porter.stem(word) for word in words]
    return stemmed_words

def remove_punctuation(text):
    text_processed = "".join([char for char in text if char not in string.punctuation])
    return text_processed


#Lower text: text = text.lower()
#Tokenization Code:  words = word_tokenize(text)
#Parts Of Speech Tagging: words = pos_tag(words)

In [27]:
custom_stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't",'no','nor','not'])
#Text preprocessing
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    return phrase


def clean_text(text):
    text = str(text)
    #lower case text
    text = text.lower().strip()

    #remove http links
    text = re.sub(r"http\S+", "", text)
    #expanding some words
    text = decontracted(text)
    #remove special character
    #text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = re.sub('[^A-Za-z]+', ' ', text)
    #remove numbers
    #text = re.sub("\S*\d\S*", "", text).strip()
    
    #text = text.strip()
    return text


def spacy_process(text):
    #Cleaning the text
    text = clean_text(text)
    doc = nlp(text)
    #print(text)
    #Tokenization and lemmatization are done with the spacy nlp pipeline commands
    lemma_list = []
    for token in doc:
        if len(token) > 1:
            lemma_list.append(token.lemma_.lower())
    #print("Tokenize+Lemmatize:")
    #print(lemma_list)
    
    #Filter the stopword
    filtered_sentence =[] 
    for word in lemma_list:
        if word not in custom_stopwords:
            filtered_sentence.append(word)
    
    #Remove punctuation
    punctuations="?:!.,;"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)

    updated_text = " ".join(filtered_sentence)
    return updated_text

first_row = fake_df.iloc[0]
print(spacy_process(first_row['text']))

df['text'] = df['text'].progress_apply(lambda x: spacy_process(x))
df.to_excel("processed_news.xlsx", index = False)
df



donald trump wish americans happy new year leave instead give shout enemy hater dishonest fake news medium former reality show star one job country rapidly grow strong smart want wish friend supporter enemy hater even dishonest fake news medium happy healthy new year president angry pant tweet great year america country rapidly grow strong smart want wish friend supporter enemy hater even dishonest fake news medium happy healthy new year great year america donald trump realdonaldtrump december trump tweet go welll expect kind president send new year greeting like despicable petty infantile gibberish trump lack decency win even allow rise gutter long enough wish american citizen happy new year bishop talbert swan talbertswan december one like calvin calvinstowell december impeachment would make great year america also accept regain control congress miranda yaver mirandayaver december hear talk include many people hate wonder hate alan sandoval alansandoval december use word hater new ye

  0%|          | 0/44940 [00:00<?, ?it/s]

,text,class
0,washington reuters senate vote wednesday wheth...,1
1,trump transition hum right along complete ever...,0
2,spew falsehood become favorite pastime democra...,0
3,feel like drug obama rhetoric greg gutfeldyes ...,0
4,ryan never make secret desire welcome immigran...,0
...,...,...
44935,strasbourg reuters european commission chief j...,1
44936,aftermath hurricane katrina dozen year ago mex...,0
44937,asian student reporter assignment espn cameram...,0
44938,armed civilian protect defend us military recr...,0


In [ ]:
#Text Vectorization, Hyperparamter tuning

Model Bulding:

used 5 fold cross validation for the below models. Generally Naive Bayes is used for baselining.

Naive Bayes

Logistic Regression

Support Vector Classifier

Decision Tree

K Nearest Neighbor

Random Forest

Extreme Gradient Boosting

Soft Voting Classifier - All Models

In [27]:
#Train Test Split
# We will use StratifiedShuffleSplit to split the data Taking into consideration that we will get the same ratio on the target column

df = pd.read_excel("processed_news.xlsx")
df = df[df['text'].notna()]

df = df.iloc[:3000]
X = df['text']
y = df['class']
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train, test in sss.split(X, y):
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]
    
print('X_train shape', X_train.shape)
print('y_train shape', y_train.shape)
print('X_test shape', X_test.shape)
print('y_test shape', y_test.shape)

# almost same ratio
print('\nratio of target in y_train :',y_train.value_counts().values/ len(y_train))
print('ratio of target in y_test :',y_test.value_counts().values/ len(y_test))
print('ratio of target in original_data :',df['class'].value_counts().values/ len(df))

X_train shape (2400,)
y_train shape (2400,)
X_test shape (600,)
y_test shape (600,)

ratio of target in y_train : [0.5225 0.4775]
ratio of target in y_test : [0.52333333 0.47666667]
ratio of target in original_data : [0.52266667 0.47733333]


In [28]:


vectorization = TfidfVectorizer()
X_train_vectors = vectorization.fit_transform(X_train)
X_test_vectors = vectorization.transform(X_test)

X_train_vectors = X_train_vectors.todense()
X_test_vectors = X_test_vectors.todense()

X_train_vectors = np.array(X_train_vectors)
X_test_vectors = np.array(X_test_vectors)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [29]:


def loss(y_true, y_pred, return_flag=False):
    pre = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    loss = log_loss(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    
    if return_flag:
        return pre, rec, f1, loss, acc
    else:
        print('  pre: %.3f\n  rec: %.3f\n  f1: %.3f\n  loss: %.3f\n  acc: %.3f' % (pre, rec, f1, loss, acc))


def simple_models(models, X_train, y_train):
    for name, model in models.items():
        cv = cross_val_score(model,X_train, y_train, cv =5)
        print("*********************** "+name+" *************************")
        print(cv)
        print(cv.mean())



In [31]:
models = {
    'GaussianNaiveBayes': GaussianNB(),
    'LogisticRegression': LogisticRegression(max_iter = 2000,random_state=42),
    #'SVC': SVC(random_state=42),
    'DecisionTreeClassifier': DecisionTreeClassifier(max_depth=5, random_state=42),
    'KNeighborsClassifier': KNeighborsClassifier(),
    'RandomForestClassifier':RandomForestClassifier(random_state = 42)
}
#Add XGBoost, voting classifiers
#voting_clf = VotingClassifier(estimators = [('lr',lr_model),('rf',rf_model),('xgb',xgb_model)], voting = 'soft') 

simple_models(models, X_train_vectors, y_train)
#After checking Cross Validation scores then Fit the train data to a model and predict output.


*********************** GaussianNaiveBayes *************************
[0.7875     0.84583333 0.82083333 0.775      0.75833333]
0.7975
*********************** LogisticRegression *************************
[0.97083333 0.95625    0.94791667 0.95833333 0.95625   ]
0.9579166666666667
*********************** DecisionTreeClassifier *************************
[0.99583333 0.98541667 0.97708333 0.99375    0.98541667]
0.9875
*********************** KNeighborsClassifier *************************
[0.82291667 0.83541667 0.8375     0.8125     0.82083333]
0.8258333333333333
*********************** RandomForestClassifier *************************
[0.97291667 0.97708333 0.96666667 0.9625     0.97708333]
0.9712499999999998
